# BEL Graph RAG Example

This notebook demonstrates how to use Graph RAG (Retrieval Augmented Generation) with BEL graphs to generate hypotheses about experimental observations.

In [2]:
import os
import json
from pathlib import Path
import sys
from ndex2 import Ndex2
import ndex2
from ndex2.cx2 import CX2Network

# Add parent directory to path to import textToKnowledgeGraph
sys.path.append('..')

import sys
# print(sys.executable)

# for key, value in os.environ.items():
#     print(f"{key}: {value}")

# Get NDEx account and password from environment variables
ndex_account = os.environ.get('NDEX_ACCOUNT')
ndex_password = os.environ.get('NDEX_PASSWORD')

# Verify credentials are available
if not ndex_account or not ndex_password:
    raise ValueError("NDEx credentials not found in environment variables. "
                    "Please set NDEX_ACCOUNT and NDEX_PASSWORD.")

ndex_client = Ndex2(username=ndex_account, password=ndex_password)


## Papers to build our knowledge graph

In [ ]:
relevant_papers = {
    "paper1": {
        "pmcid": "PMC5572400"
    },
}

## Process the relevant papers to CX2 files

In [4]:
# Process each paper to generate CX2 knowledge graphs and save to files
from python_scripts.bel_processing import process_paper_to_bel_cx2
paper_ndex_ids = []
for paper_id, paper in relevant_papers.items():
    cx2_network = process_paper_to_bel_cx2(paper_id)



ModuleNotFoundError: No module named 'pub'

## Merge Graphs

In [ ]:
def merge_cx2(cx2_graphs):
    merged_graph = CX2Network()
    node_map = {}  # Maps (node_name, node_type) to node ID in merged graph
    
    # First, merge all nodes
    for graph in cx2_graphs:
        for node_id, node_data in graph.get_nodes().items():  # Changed to .items() based on docs
            # Create a tuple of node attributes that define uniqueness
            node_name = node_data.get('name', '')
            node_type = node_data.get('type', '')  # Changed 'r' to 'type' as more standard
            node_key = (node_name, node_type)
            
            if node_key not in node_map:
                # Create new node using add_node() as documented
                new_node_id = merged_graph.add_node(attributes=node_data)
                node_map[node_key] = new_node_id
    
    # Then, merge all edges
    for graph in cx2_graphs:
        for edge_id, edge_data in graph.get_edges().items():  
            # Get source and target directly from edge data
            source_id = edge_data.get('source')  # Changed 's' to 'source'
            target_id = edge_data.get('target')  # Changed 't' to 'target'
            
            # Get source and target nodes
            source_node = graph.get_node(source_id)
            target_node = graph.get_node(target_id)
            
            # Get corresponding nodes in merged graph
            source_key = (source_node.get('name', ''), source_node.get('type', ''))
            target_key = (target_node.get('name', ''), target_node.get('type', ''))
            
            merged_source = node_map[source_key]
            merged_target = node_map[target_key]
            
            # Create edge using add_edge() as documented
            merged_graph.add_edge(source=merged_source, 
                                target=merged_target, 
                                attributes=edge_data)
    
    return merged_graph

## Example Experiment

Let's use an example experiment studying the effects of oxidative stress on cell death pathways.

In [18]:
example_experiment = {
    "experiment": """
    Human endothelial cells were treated with hydrogen peroxide (H2O2) at varying concentrations 
    (0, 100, 200, 500 μM) for 24 hours. Cell viability, apoptosis markers, and oxidative stress 
    indicators were measured.
    """,
    
    "measurements": """
    1. Cell viability decreased dose-dependently with H2O2 treatment
    2. Caspase-3 activity increased 3-fold at 200 μM H2O2
    3. Intracellular ROS levels increased 5-fold at 200 μM H2O2
    4. Bcl-2 protein levels decreased by 50% at 200 μM H2O2
    5. Cytochrome c was detected in cytoplasmic fractions at 200 μM H2O2
    """
}

example_experiment_2 = {
    "experiment": """
    Recent studies have revealed intriguing connections between cellular metabolism and circadian rhythms, 
    though the molecular mechanisms linking these processes remain poorly understood. Our network analysis 
    suggests that SIRT1, a NAD+-dependent deacetylase known to respond to cellular energy status, may serve 
    as a key mediator through its interactions with core clock proteins. Specifically, SIRT1's direct 
    deacetylation of both BMAL1 at K537 and PER2 suggests a mechanism whereby metabolic state could 
    directly influence circadian timing. We hypothesize that SIRT1 functions as a metabolic sensor that 
    fine-tunes circadian rhythms by modulating the acetylation status of clock proteins in response to 
    cellular NAD+ levels. This would create a feedback loop where daily oscillations in metabolism influence 
    clock protein activity through SIRT1, which in turn affects metabolic gene expression through clock-controlled 
    transcription. Furthermore, since SIRT1 decreases TIP60 activity, and TIP60 is a known acetyltransferase, 
    we predict that SIRT1 suppression of TIP60 would lead to enhanced deacetylation of clock proteins through an 
    indirect mechanism independent of SIRT1's direct deacetylase activity. This dual mechanism of direct and 
    indirect deacetylation control could explain the observed disruption of both circadian rhythms and metabolism 
    in aging and metabolic diseases. Testing this hypothesis will require careful analysis of how manipulation of 
    cellular NAD+ levels affects SIRT1-mediated deacetylation of clock proteins and subsequent changes in 
    circadian period and amplitude. """

}

## Use an LLM to extract the entities from the example experiment, then query the KG in NDEx

In [29]:
import os
from openai import OpenAI
from typing import List

# Get API key from environment
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY not found in environment variables")

# Initialize OpenAI client
client = OpenAI(api_key=OPENAI_API_KEY)

# Template for extracting gene entities
entity_prompt_template = """
Interpret this text to extract all genes/proteins mentioned and output them as a whitespace-separated list of human gene symbols.
<example>
TP53 AKT1 MTOR
</example>
Only output that list, nothing else.
<text>
{text}
</text>
"""

def query_llm(prompt: str) -> str:
    """
    Query OpenAI's GPT model.
    """
    try:
        completion = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "user", "content": prompt}
            ],
            temperature=0.7,
            max_tokens=1000
        )
        return completion.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error querying OpenAI: {str(e)}")
        return ""

def get_entities(text: str) -> List[str]:
    """
    Extract gene entities from text using LLM.
    """
    prompt = entity_prompt_template.format(text=text)
    response = query_llm(prompt)
    if response:
        return response
    return ""

kg_query_string = get_entities(example_experiment)

print(kg_query_string)

# kg_query_string = "SIRT1 TP53"

kg_network_id = "7ce89103-a372-11ef-99aa-005056ae3c32"

nice_kg_query_network = ndex2.create_nice_cx_from_raw_cx(ndex_client.get_neighborhood(kg_network_id, kg_query_string, search_depth=1))

# convert the network to a string containing BEL statements and supporting evidence
knowledge_graph = ""

for edge_id, edge_obj in nice_kg_query_network.get_edges():
    knowledge_graph += nice_kg_query_network.get_edge_attribute_value(edge_obj, "bel_expression")
    knowledge_graph += "\n"

knowledge_graph

CASP3 BCL2 CYC


'a(CHEBI:ATP) increases act(p(HGNC:CASP3)\n'

## Hypothesis Generation Prompt

In [30]:
PROMPT_TEMPLATE = """
You are an expert biologist tasked with analyzing experimental data and proposing mechanistic hypotheses.

EXPERIMENT DESCRIPTION:
{experiment}

MEASUREMENTS:
{measurements}

BEL FORMAT GUIDELINES:
BEL is a language for representing biological knowledge in a computable form. Key aspects:
- Entities are represented with functions like p() for proteins, a() for abundances, bp() for biological processes
- Relationships between entities use operators like increases, decreases, directlyIncreases, association
- Entities must use standard namespaces (HGNC for human genes, CHEBI for chemicals, etc.)

EXISTING KNOWLEDGE (Optional):
{knowledge_graph}

TASK:
Propose a hypothesis explaining the experimental observations as a BEL graph. Your hypothesis should:
1. Be consistent with the experimental data
2. Use proper BEL syntax and namespaces
3. Focus on mechanistic relationships
4. Incorporate existing knowledge when provided

Output your hypothesis as a list of BEL statements that form a connected graph.
"""

## Hypothesis Generation Function

In [32]:
def generate_hypothesis(experiment_desc, measurements, knowledge_graph=None):
    """Generate a hypothesis as a BEL graph based on experimental observations.
    
    Args:
        experiment_desc (str): Description of the experimental setup
        measurements (str): Observed experimental measurements
        knowledge_graph (str, optional): Existing knowledge in BEL format
        
    Returns:
        str: Path to the saved CX2 file containing the hypothesis graph
    """
    # Format the prompt
    prompt = PROMPT_TEMPLATE.format(
        experiment=experiment_desc,
        measurements=measurements,
        knowledge_graph=knowledge_graph if knowledge_graph else "No prior knowledge provided"
    )

    return query_llm(prompt)
    


## Generate a hypothesis

In [34]:
hypothesis = generate_hypothesis(example_experiment['experiment'], example_experiment['measurements'], knowledge_graph)
print(hypothesis)

1. a(CHEBI:"hydrogen peroxide") increases a(CHEBI:"reactive oxygen species")
2. a(CHEBI:"hydrogen peroxide") decreases p(HGNC:BCL2)
3. a(CHEBI:"reactive oxygen species") increases act(p(HGNC:CASP3))
4. p(HGNC:BCL2) decreases act(p(HGNC:CASP3))
5. act(p(HGNC:CASP3)) increases a(CHEBI:"cytochrome c")
6. a(CHEBI:"cytochrome c") decreases bp(GO:"cell viability")
7. act(p(HGNC:CASP3)) increases bp(MESH:"Apoptosis")
8. a(CHEBI:"hydrogen peroxide") directlyDecreases bp(GO:"cell viability")
